In [1]:
import orjson
import ijson

In [2]:
data_path = "output/Qwen3-Coder-30B-A3B/Scenario.codegeneration_10_0.2_eval_all.json"
out_path = "output/Qwen3-Coder-30B-A3B/Scenario.codegeneration_10_0.2_eval_all.jsonl"
jsonlines = "output/Qwen3-Coder-30B-A3B/codegeneration_10_0.2_eval_all.jsonl"

In [6]:


# convert to jsonl
def clean_logprobs(logprobs_list):
    for sequence in logprobs_list:
        for token in sequence:
            for token_id in token:
                token[token_id]['logprob'] = str(token[token_id]['logprob'])
    return logprobs_list

with open(data_path, "rb") as infile, open(out_path, "wb") as outfile:
    for item in ijson.items(infile, "item"):  # iterate over each object in array
        item['pass@1'] =str(item['pass@1'])
        item['logprobs_list'] = clean_logprobs(item['logprobs_list'])
        line = orjson.dumps(item) + b"\n"
        outfile.write(line)

    


In [ ]:
from numpy import mean


def top20_logprob_sum_f(sample: dict):
    for sequence in sample["logprobs_list"]:
        top20 = []
        count = 0                               #count = Token-Nr. (beginnend mit 0)
        for token in sequence:
            top20.append(0)
            for k in token.values():
                if (k["rank"]>= 1) and (k["rank"]<= 20):
                    top20[count]+= float (k["logprob"])
            count += 1

    return top20
    


def prepare_sample(sample: dict):
    logprobs = [
                [
                    k["logprob"] for d in sequence for k in filter(lambda x: x["rank"] == 1, d.values())
                ] for sequence in sample["logprobs_list"]
            ]
    top20_logprob_sum = top20_logprob_sum_f(sample)
    return {
        "id": sample["question_id"],
        "name": sample["question_title"],
        "prompt": sample["question_content"],
        "program": sample["code_list"],
        "language": 'python',
        "output_size": [len(output) for output in sample["output_list"]],
        "token_count": [len(logprobs) for logprobs in sample["logprobs_list"]],
        "is_correct": sample["graded_list"],
        "token_logprobs": logprobs,
        'cumulative_logprob': [sum([float(x) for x in l]) for l in logprobs],
        "top_logprobs_first_token": top20_logprob_sum[0],
        "top_logprobs_first_20_tokens": float (mean(top20_logprob_sum[0:19])),
        "top_logprobs_last_token": top20_logprob_sum[-1],
        "top_logprobs_last_20_tokens": float (mean(top20_logprob_sum[(len(top20_logprob_sum)-21):(len(top20_logprob_sum)-1)])),
        "difficulty": sample["difficulty"],
    }

In [ ]:
i=0
results = []
with open(jsonlines, "rb") as f:   
    for line in f:
        if i>=20:
            break
        obj = orjson.loads(line)
        sample = prepare_sample(obj)
        results.append(sample)
        print(sample)
        i+=1

In [27]:
evaluation_data_path = "output/Qwen3-Coder-30B-A3B/preprocessed/codegeneration_10_0.2.jsonl"

with open(evaluation_data_path, "wb") as outfile:
    for item in results:     
        line = orjson.dumps(item) + b"\n"
        outfile.write(line)

# Groups
- loc
- cyclomatic complexity
- len output
- len code
- difficulty
- len prompt
- language

(identifier count, etc)

# Metrics
- mean logprob
@ Katharina:
- sum top-20 logprobs (last n tokens, last token, last code token, first token)
    - 1) last 20 tokens (Konfigurierbar für n)
    - 2) last token
    - 3) first 20 tokens
    - 4) last 20 code tokens
    - 5) last code token
    - 6) first code token